In [10]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeReals, Suffix, value
from pyomo.opt import SolverFactory
import pandas as pd

# 1. read all input data from files

relevant tsv-files are:
- load
- duration
- availability
- tech_data

In [11]:
load = pd.read_csv("load.tsv",  sep="\s+", names=["value"] )
load = load.T
load

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
value,82115,73169,68729,63442,60430,57013,52048,48701,43981,40498


In [12]:
tech_data = pd.read_csv("tech_data.tsv", sep="\s+", header=0, index_col=False, skiprows=[1], decimal=".") #workaround bc read csv shifts columnnames
tech_data.set_index("tech", inplace=True)
tech_data

,Cap,ETA_EL,Fuel_P,c_var_other,EMF
tech,,,,,
CCGT,29555,0.54,12.8,1.5,0.2048
GT_GasOil,4400,0.28,12.8,1.5,0.2048
Hydro,5256,1.00,0.0,1.5,0.0000
Coal,22458,0.42,7.4,2.6,0.3420
Lignite,21067,0.37,3.4,3.0,0.3996
Nuclear,8114,0.33,1.8,0.7,0.0000
Wind,61114,1.00,0.0,1.4,0.0000
Solar,46471,1.00,0.0,1.0,0.0000


In [13]:
availability = pd.read_csv("availability.tsv", sep="\s+")
availability

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
CCGT,19629,19629,19629,19629,19629,19629,19629,19629,19629,19629
GT_GasOil,2980,2980,2980,2980,2980,2980,2980,2980,2980,2980
Hydro,4012,4012,4012,4012,4012,4012,4012,4012,4012,4012
Coal,19564,19564,19564,19564,19564,19564,19564,19564,19564,19564
Lignite,17687,17687,17687,17687,17687,17687,17687,17687,17687,17687
Nuclear,7610,7610,7610,7610,7610,7610,7610,7610,7610,7610
Wind,11480,9858,11332,14498,20590,21012,6314,16072,21238,12054
Solar,10538,2078,3920,7641,6970,0,14810,1307,0,0


In [14]:
duration = pd.read_csv("duration.tsv", sep="\s+", names=["value"])
duration = duration.T
duration

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
value,102,962,962,962,962,962,962,962,962,962


# 2. set up calculations
if thats necessary.

In [15]:
tech_data["costs_el_no_co2"] = tech_data.Fuel_P / tech_data.ETA_EL + tech_data.c_var_other
tech_data.costs_el_no_co2

tech
CCGT         25.203704
GT_GasOil    47.214286
Hydro         1.500000
Coal         20.219048
Lignite      12.189189
Nuclear       6.154545
Wind          1.400000
Solar         1.000000
Name: costs_el_no_co2, dtype: float64

In [16]:
tech_data["emissions_el"] = tech_data.EMF / tech_data.ETA_EL
tech_data.emissions_el

tech
CCGT         0.379259
GT_GasOil    0.731429
Hydro        0.000000
Coal         0.814286
Lignite      1.080000
Nuclear      0.000000
Wind         0.000000
Solar        0.000000
Name: emissions_el, dtype: float64

In [17]:
co2_price = 50
tech_data["costs_el_w_co2"] = (tech_data.Fuel_P / tech_data.ETA_EL) + tech_data.c_var_other + (co2_price * tech_data.EMF / tech_data.ETA_EL)
tech_data

,Cap,ETA_EL,Fuel_P,c_var_other,EMF,costs_el_no_co2,emissions_el,costs_el_w_co2
tech,,,,,,,,
CCGT,29555,0.54,12.8,1.5,0.2048,25.203704,0.379259,44.166667
GT_GasOil,4400,0.28,12.8,1.5,0.2048,47.214286,0.731429,83.785714
Hydro,5256,1.00,0.0,1.5,0.0000,1.500000,0.000000,1.500000
Coal,22458,0.42,7.4,2.6,0.3420,20.219048,0.814286,60.933333
Lignite,21067,0.37,3.4,3.0,0.3996,12.189189,1.080000,66.189189
Nuclear,8114,0.33,1.8,0.7,0.0000,6.154545,0.000000,6.154545
Wind,61114,1.00,0.0,1.4,0.0000,1.400000,0.000000,1.400000
Solar,46471,1.00,0.0,1.0,0.0000,1.000000,0.000000,1.000000


# 3. configuration of model

defined function to solve each timestep

In [18]:
def model_solve (timestep, co2=False):
    model = ConcreteModel()

    model.x = Var(tech_data.index.values, domain=NonNegativeReals) #dispatch variable

    model.balance = Constraint(expr = sum(model.x[gen] for gen in tech_data.index.values) == load.loc[:, timestep].value) # meet demand

    def avail_cap(model,gen):
        return model.x[gen] <= availability.loc[gen, timestep] # availability

    model.cap_limits = Constraint(tech_data.index.values, rule=avail_cap)
    
    if co2:
        model.objective = Objective(expr = sum([tech_data.costs_el_w_co2[gen]*model.x[gen] for gen in tech_data.index.values]))
    else:
        model.objective = Objective(expr = sum([tech_data.costs_el_no_co2[gen]*model.x[gen] for gen in tech_data.index.values]))

    opt = SolverFactory("glpk")

    model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)

    results = opt.solve(model,suffixes=["dual"])

    #results.write()
    return model, results

# 4. Post modeling

## 4.1 create dispatch dataframe from result
-> Toggle co2_price!

In [90]:
co2_price = True # toggle co2 False and True for co2-price
result_df = pd.DataFrame()
for t in load.columns.values:
    model, results = model_solve(t, co2_price)     
    for gen in model.x: 
        result_df.loc[gen,t] = model.x[gen].value
result_df = result_df.reindex(tech_data.index) #same order as tech_data
result_df
    

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
tech,,,,,,,,,,
CCGT,19629.0,19629.0,19629.0,19629.0,19629.0,19629.0,19302.0,19629.0,11121.0,16822.0
GT_GasOil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hydro,4012.0,4012.0,4012.0,4012.0,4012.0,4012.0,4012.0,4012.0,4012.0,4012.0
Coal,19564.0,19564.0,19564.0,10052.0,1619.0,4750.0,0.0,71.0,0.0,0.0
Lignite,9282.0,10418.0,2662.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Nuclear,7610.0,7610.0,7610.0,7610.0,7610.0,7610.0,7610.0,7610.0,7610.0,7610.0
Wind,11480.0,9858.0,11332.0,14498.0,20590.0,21012.0,6314.0,16072.0,21238.0,12054.0
Solar,10538.0,2078.0,3920.0,7641.0,6970.0,0.0,14810.0,1307.0,0.0,0.0


## 4.2 calculate co2 emissions

In [91]:
co2_df = result_df.mul(tech_data.EMF, axis=0) #multiply with emission factors
co2_df

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
tech,,,,,,,,,,
CCGT,4020.0192,4020.0192,4020.0192,4020.0192,4020.0192,4020.0192,3953.0496,4020.0192,2277.5808,3445.1456
GT_GasOil,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
Hydro,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
Coal,6690.8880,6690.8880,6690.8880,3437.7840,553.6980,1624.5000,0.0000,24.2820,0.0000,0.0000
Lignite,3709.0872,4163.0328,1063.7352,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
Nuclear,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
Wind,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
Solar,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [92]:
co2_df.sum() # co2 emissions in each timestep

t1     14419.9944
t2     14873.9400
t3     11774.6424
t4      7457.8032
t5      4573.7172
t6      5644.5192
t7      3953.0496
t8      4044.3012
t9      2277.5808
t10     3445.1456
dtype: float64

In [93]:
co2_df.sum().sum() # annual co2 emissions

72464.69360000001

## 4.3 calculate electricity price

In [94]:
price_df = result_df > 0
if co2_price:
    price_df = price_df.mul(tech_data.costs_el_w_co2, axis=0)
else:
    price_df = price_df.mul(tech_data.costs_el_no_co2, axis=0)
price_df #price for each gen each timestep

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
tech,,,,,,,,,,
CCGT,44.166667,44.166667,44.166667,44.166667,44.166667,44.166667,44.166667,44.166667,44.166667,44.166667
GT_GasOil,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Hydro,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000
Coal,60.933333,60.933333,60.933333,60.933333,60.933333,60.933333,0.000000,60.933333,0.000000,0.000000
Lignite,66.189189,66.189189,66.189189,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Nuclear,6.154545,6.154545,6.154545,6.154545,6.154545,6.154545,6.154545,6.154545,6.154545,6.154545
Wind,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000
Solar,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000


In [95]:
elec_price = pd.DataFrame(price_df.max(axis=0), columns=["value"])
elec_price = elec_price.T #marketprice each timestep
elec_price

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
value,66.189189,66.189189,66.189189,60.933333,60.933333,60.933333,44.166667,60.933333,44.166667,44.166667


In [96]:
annual_price = elec_price.mul(load, axis=1).sum(axis=1) / load.sum(axis=1) #average electricity price weighted by amount of energy
annual_price = annual_price[0]
annual_price

59.04945995347521

## 4.4 calculate contribution

In [107]:
annual_gen = result_df.sum(axis=1)
annual_revenue = result_df.mul(elec_price.values, axis=1).sum(axis=1)
if co2_price:
    annual_var_cost = annual_gen.mul(tech_data.costs_el_w_co2)
annual_contribution = annual_revenue - annual_var_cost
annual_contribution

tech
CCGT         2.613292e+06
GT_GasOil    0.000000e+00
Hydro        2.245921e+06
Coal         3.084767e+05
Lignite      0.000000e+00
Nuclear      3.905874e+06
Wind         8.107119e+06
Solar        2.671286e+06
dtype: float64

# shadow price

In [88]:
model.dual[model.balance]

44.1666666666667